In [1]:
from pathlib import Path
import geopandas as gpd
import folium
# from IPython.display import display

path_no_hazard_OD_result = Path(r"C:\Users\hauth\OneDrive - Stichting Deltares\Documents\tempo\RACE cases\hackathon july\multi_link_origin_closest_destination_optimal_routes_without_hazard.gpkg")
path_with_hazard_OD_result = Path(r"C:\Users\hauth\OneDrive - Stichting Deltares\Documents\tempo\RACE cases\hackathon july\multi_link_origin_closest_destination_optimal_routes_with_hazard.gpkg")
path_origins = Path(r"C:\Users\hauth\OneDrive - Stichting Deltares\Documents\tempo\RACE cases\hackathon july\multi_link_origin_closest_destination_origins.gpkg")
path_destinations = Path(r"c:\Users\hauth\OneDrive - Stichting Deltares\Documents\tempo\RACE cases\hackathon july\multi_link_origin_closest_destination_destinations.gpkg")

In [66]:
from pathlib import Path
import geopandas as gpd
import folium
# from IPython.display import display
root_dir = Path(r"C:\Users\hauth\OneDrive - Stichting Deltares\Desktop\projects\RACE hackathon\july hackathon\root_dir\output\multi_link_origin_closest_destination_save")
path_no_hazard_OD_result = root_dir.joinpath("origin_closest_destination_optimal_routes_without_hazard.gpkg")
path_with_hazard_OD_result = root_dir.joinpath("origin_closest_destination_optimal_routes_with_hazard.gpkg")

path_origins = root_dir.joinpath("origin_closest_destination_origins.gpkg")
path_destinations = root_dir.joinpath(r"origin_closest_destination_destinations.gpkg")

In [67]:
# Open the gdf
optimal_routes_no_hazard_gdf = gpd.read_file(path_no_hazard_OD_result)
optimal_routes_with_hazard_gdf = gpd.read_file(path_with_hazard_OD_result)
origins_gdf = gpd.read_file(path_origins)
destinations_gdf = gpd.read_file(path_destinations)



# Get the all the unique destinations:
destinations = optimal_routes_no_hazard_gdf["destination"].unique()

# keep only ctaegory hospital
# destinations_gdf = destinations_gdf[destinations_gdf["category"] == "education"]





In [68]:
# clean destination gdf to account for combined destinations "B_5,B_6"
temp_gdf = gpd.GeoDataFrame()
for dest in destinations:
    if "," in dest:
        dests = dest.split(",")
        temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dests[0]])
        # rename the destination id to the combined destination id
        temp_gdf.loc[temp_gdf["d_id"] == dests[0], "d_id"] = dest
        
    else:
        temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dest])

destinations_gdf = temp_gdf


C:\Users\hauth\AppData\Local\Temp\ipykernel_45060\6077222.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dest])
C:\Users\hauth\AppData\Local\Temp\ipykernel_45060\6077222.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dest])
C:\Users\hauth\AppData\Local\Temp\ipykernel_45060\6077222.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dest])
C:\Users\hauth\AppData\Local\Temp\ipykernel_45060\6077222.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

In [71]:
total_number_of_population_no_hazard = 0
total_number_of_population_with_hazard = 0
for destination in destinations:
    optimal_routes_per_destination_no_hazard_gdf = optimal_routes_no_hazard_gdf[optimal_routes_no_hazard_gdf['destination'] == destination] # filter on destination B4
    optimal_routes_per_destination_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['destination'] == destination] # filter on destination B4
    #get the id of all the origins associated with the destination 
    list_origins_id_no_hazard = optimal_routes_per_destination_no_hazard_gdf['origin'].unique()
    list_origins_id_with_hazard = optimal_routes_per_destination_with_hazard_gdf['origin'].unique()
    
    gdf_origins = gpd.read_file(path_origins)
    # filter the origins gdf to only include the origins associated with the destination B4
    gdf_origins_filtered_no_hazard = gdf_origins[gdf_origins['o_id'].isin(list_origins_id_no_hazard)]
    gdf_origins_filtered_with_hazard = gdf_origins[gdf_origins['o_id'].isin(list_origins_id_with_hazard)]
    

    # count the column POPULATION:
    nb_population_no_hazard = int(gdf_origins_filtered_no_hazard['POPULATION'].sum())
    nb_population_with_hazard = int(gdf_origins_filtered_with_hazard['POPULATION'].sum())
    
    total_number_of_population_no_hazard += nb_population_no_hazard
    total_number_of_population_with_hazard += nb_population_with_hazard
    diff_population = nb_population_no_hazard - nb_population_with_hazard
    if nb_population_no_hazard == 0:
        relative_diff_population = 0
    else:
        relative_diff_population = round(diff_population / nb_population_no_hazard * 100, 1)
    print(f"Destination: {destination}, No hazard: {nb_population_no_hazard}, With hazard {nb_population_with_hazard}")
    # add count population to destination gdf
    destinations_gdf.loc[destinations_gdf['d_id'] == destination, 'population_no_hazard'] = nb_population_no_hazard
    destinations_gdf.loc[destinations_gdf['d_id'] == destination, 'population_with_hazard'] = nb_population_with_hazard
    destinations_gdf.loc[destinations_gdf['d_id'] == destination, 'diff_population'] = diff_population
    destinations_gdf.loc[destinations_gdf['d_id'] == destination, 'relative_diff_population'] = relative_diff_population
    
print("No hazard: ", total_number_of_population_no_hazard)
print("With hazard: ", total_number_of_population_with_hazard)
print("population with no access: ", total_number_of_population_no_hazard - total_number_of_population_with_hazard)

print(destinations_gdf)
print(destinations_gdf.columns)


destinations_gdf



Destination: B_289, No hazard: 85689, With hazard 85689
Destination: B_534, No hazard: 42946, With hazard 42946
Destination: B_592, No hazard: 184266, With hazard 184266
Destination: B_550, No hazard: 38459, With hazard 38459
Destination: B_549, No hazard: 115499, With hazard 115499
Destination: B_509, No hazard: 12918, With hazard 12918
Destination: B_553,B_555, No hazard: 177184, With hazard 177184
Destination: B_552, No hazard: 73321, With hazard 73321
Destination: B_328, No hazard: 88935, With hazard 88935
Destination: B_330, No hazard: 16009, With hazard 16009
Destination: B_475, No hazard: 108364, With hazard 108364
Destination: B_441, No hazard: 32598, With hazard 32598
Destination: B_476, No hazard: 46459, With hazard 46459
Destination: B_511, No hazard: 48150, With hazard 48150
Destination: B_443, No hazard: 97366, With hazard 97366
Destination: B_446, No hazard: 64330, With hazard 64330
Destination: B_487, No hazard: 95814, With hazard 95814
Destination: B_517, No hazard: 754

,OBJECTID,POPULATION,d_id,category,EV1_ma,EV1_ma_PD1,EV1_ma_PD2,EV1_ma_PD3,EV1_ma_PD4,geometry,population_no_hazard,population_with_hazard,diff_population,relative_diff_population
289,289,None,B_289,clinic,0,85689.148438,0.0,0.0,0.000000e+00,POINT (35.47918 -16.21720),85689.0,85689.0,0.0,0.0
534,534,None,B_534,clinic,0,42946.328857,0.0,0.0,0.000000e+00,POINT (33.45491 -17.84092),42946.0,42946.0,0.0,0.0
592,592,None,B_592,clinic,0,184266.914062,0.0,0.0,0.000000e+00,POINT (32.77324 -20.57782),184266.0,184266.0,0.0,0.0
550,550,None,B_550,clinic,0,38459.324219,0.0,0.0,0.000000e+00,POINT (33.28283 -21.31794),38459.0,38459.0,0.0,0.0
549,549,None,B_549,clinic,0,115499.593750,0.0,0.0,0.000000e+00,POINT (33.27235 -20.95457),115499.0,115499.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,343,None,B_343,doctors,0,0.000000,0.0,0.0,1.510071e+05,POINT (34.75763 -19.63004),151007.0,151007.0,0.0,0.0
235,235,None,B_235,doctors,0,0.000000,0.0,0.0,1.218047e+06,POINT (35.73790 -17.96026),1218046.0,1218046.0,0.0,0.0
279,279,None,B_279,doctors,0,0.000000,0.0,0.0,2.034432e+05,POINT (35.31170 -17.86690),203443.0,203443.0,0.0,0.0
359,359,None,B_359,doctors,0,0.000000,0.0,0.0,2.029742e+05,POINT (35.03890 -17.68440),202974.0,202974.0,0.0,0.0


In [73]:
# make pretty map
import folium

base_map = folium.Map(location=[-19.84505, 34.8734], zoom_start=13, tiles="CartoDB dark_matter")

destinations_group = folium.FeatureGroup(name='Destinations')
for _, row in destinations_gdf.iterrows():
    if row['population_with_hazard'] ==0:
        color='red'
        hover= 'Unreachable destination'
    else:
        if row['population_no_hazard'] - row['population_with_hazard'] > 0:
            color='green'
            # hover= f'Popultation no hazard {row["population_no_hazard"]}, <br> population with hazard {row["population_with_hazard"]}'
            hover_strings = f"<b>Destination</b> {row['d_id']} <br>" \
                            f"<b>Population no hazard:</b> {row['population_no_hazard']} <br>" \
                            f"<b>Population with hazard:</b> {row['population_with_hazard']} <br>" \
                            f"<b>Decrease: -</b> {row['diff_population']} <br>" \
                            f"<b>Relative decrease: -</b> {row['relative_diff_population']} %<br>"
            hover = folium.Popup(hover_strings, max_width=300)
        else:
            if row['diff_population'] == 0:
                color='green'
            else:
                color='yellow'
            # hover = f'Population no hazard {row["population_no_hazard"]}, population with hazard {row["population_with_hazard"]}'
            hover_strings = f"<b>Destination</b> {row['d_id']} <br>" \
                            f"<b>Population no hazard:</b> {row['population_no_hazard']} <br>" \
                            f"<b>Population with hazard:</b> {row['population_with_hazard']} <br>" \
                            f"<b>Increase: +</b> {row['diff_population']} <br>" \
                            f"<b>Relative increase: +</b> {row['relative_diff_population']} %<br>"
            hover = folium.Popup(hover_strings, max_width=300)            
            
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x], 
        radius=5,
        color=color, 
        fill=True,
        
        popup=hover
        ).add_to(base_map)


# filter origins

# # destination_display = "B_2"
# # destination_display = "B_5,B_6"
# optimal_routes_per_destination_no_hazard_gdf = optimal_routes_no_hazard_gdf[optimal_routes_no_hazard_gdf['destination'] == destination_display] # filter on destination B4
# optimal_routes_per_destination_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['destination'] == destination_display] # filter on destination B4
#     #get the id of all the origins associated with the destination 
# list_origins_id_no_hazard = optimal_routes_per_destination_no_hazard_gdf['origin'].unique()
# list_origins_id_with_hazard = optimal_routes_per_destination_with_hazard_gdf['origin'].unique()
    
#     # filter the origins gdf to only include the origins associated with the destination B4
# gdf_origins_filtered_no_hazard = gdf_origins[gdf_origins['o_id'].isin(list_origins_id_no_hazard)]
# gdf_origins_filtered_with_hazard = gdf_origins[gdf_origins['o_id'].isin(list_origins_id_with_hazard)]
    
# # gdf_origins_filtered_no_hazard.explore(m=base_map, color='green', marker_kwds={'radius': 5})
# gdf_origins_filtered_with_hazard.explore(m=base_map, color='white', marker_kwds={'radius': 2})

#     # Create a feature group for target_optimal_routes_with_hazard_gdf
# target_optimal_routes_with_hazard_map = folium.FeatureGroup(name="Optimal Routes With Hazard")
# optimal_routes_with_hazard_gdf_temps = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['destination'] == destination_display]
# optimal_routes_with_hazard_gdf_temps.explore(m=target_optimal_routes_with_hazard_map, color='red', line_kwds={'weight': 2})
# target_optimal_routes_with_hazard_map.add_to(base_map)

base_map



In [34]:
destinations_gdf

,OBJECTID,POPULATION,d_id,category,EV1_ma,EV1_ma_PD1,EV1_ma_PD2,geometry,population_no_hazard,population_with_hazard,diff_population,relative_diff_population
2,2,None,B_2,hospital,0.0,54990.600084,0.000000e+00,POINT (34.84526 -19.84216),22941.0,54148.0,-31207.0,-136.0
0,0,None,B_0,hospital,0.0,0.000000,0.000000e+00,POINT (34.85672 -19.82892),40703.0,0.0,40703.0,100.0
3,3,None,B_3,hospital,0.0,0.000000,0.000000e+00,POINT (34.86481 -19.84662),44583.0,0.0,44583.0,100.0
1,1,None,B_1,hospital,0.0,53244.691439,0.000000e+00,POINT (34.85680 -19.82894),29957.0,53244.0,-23287.0,-77.7
5,5,None,"B_5,B_6",hospital,0.0,0.000000,1.304040e+16,POINT (34.87400 -19.85082),67107.0,56358.0,10749.0,16.0
4,4,None,B_4,hospital,0.0,0.000000,5.103422e+04,POINT (34.87519 -19.84902),71077.0,51034.0,20043.0,28.2


In [22]:
# make pretty map
import folium
from folium.plugins import MarkerCluster

# Function to create a folium map with layer control
def create_map_with_layer_control():
    # Create a base map
    base_map = folium.Map(location=[0, 0], zoom_start=2, tiles="CartoDB dark_matter")

    # Create a MarkerCluster for target_destinations_gdf
    marker_cluster = MarkerCluster(name="Target Destinations").add_to(base_map)

    # Add markers for target_destinations_gdf to the MarkerCluster
    for _, row in destinations_gdf.iterrows():
        # Adjust column names based on your dataset
        folium.Marker(location=[row['geometry'].y, row['geometry'].x], popup=row['d_id']).add_to(marker_cluster)

    # Create a feature group for target_origins_gdf
    target_origins_map = folium.FeatureGroup(name="Target Origins")
    origins_gdf.explore(m=base_map, color='green', marker_kwds={'radius': 5})
    target_origins_map.add_to(base_map)

    # Create a feature group for target_optimal_routes_with_hazard_gdf
    target_optimal_routes_with_hazard_map = folium.FeatureGroup(name="Optimal Routes With Hazard")
    optimal_routes_with_hazard_gdf.explore(m=target_optimal_routes_with_hazard_map, color='red', line_kwds={'weight': 2})
    target_optimal_routes_with_hazard_map.add_to(base_map)

    # Add LayerControl to toggle layers based on d_id
    folium.LayerControl(collapsed=False).add_to(base_map)

    # Save the map to an HTML file for manual inspection
    base_map.save("map_with_layer_control.html")

    return base_map

# Display the map
create_map_with_layer_control()